<a href="https://colab.research.google.com/github/KaRtHiK-56/Race_Car_DQN_RL/blob/main/Race_Car_DQN_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gymnasium==0.29.1
!pip install gymnasium[accept-rom-license]
!apt -get install -y swig
!pip install wheel setuptools pip --upgrade
!pip install swig
!pip install gymnasium[box2d]

E: Command line option 'g' [from -get] is not understood in combination with the other options.
  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2376102 sha256=fa531488afec75cf5787badaa372679028048964f46a9821d838e5ff7237c411
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py


In [2]:
#importing the necessary libraries
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import gymnasium as gym

In [49]:
#build the AI model - creating the base neural network
class Network(nn.Module):

  def __init__(self, state_size, action_size,seed = 42):
    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.linear1 = nn.Linear(state_size,128)
    self.linear2 = nn.Linear(128,64)
    self.linear3 = nn.Linear(64,output_size)

  def forward(self, state):
    state = state.view(state.size(0), -1)  # Flatten state to (batch_size, state_size)
    x = F.relu(self.linear1(state))
    x = F.relu(self.linear2(x))
    return self.linear3(x)

In [50]:
#setting up the gymnasium environment for the car racing RL
env = gym.make("CarRacing-v2", render_mode="rgb_array")
state_shape = env.observation_space.shape
state_size = state_shape[0] * state_shape[1] * state_shape[2] #multiplying the arrays because of 3d shape of state_shape
output_size = env.action_space.shape[0]
number_actions  = env.action_space.shape[0]

print('State shape: ', state_shape)
print('State size: ', state_size)
print('output_size: ', output_size)
print('Number of actions: ', number_actions)

State shape:  (96, 96, 3)
State size:  27648
output_size:  3
Number of actions:  3


In [51]:
#initializing the hyperparameters
learning_rate = 0.00005
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size =  100000
interpolation_parameter = 0.001

In [52]:
#implementing experience replay
class ReplayBuffer(object):
  def __init__(self,capacity):
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []
    print("the device available in the local system is: ",self.device)

  def push(self,event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
        experiences = random.sample(self.memory, k = batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)

  def __len__(self):
      return len(self.memory)


In [53]:
#implementing the Deep Q-Network
class Agent():
  def __init__(self,state_size,number_actions):
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.number_actions = number_actions
    self.local_qnetwork = Network(state_size,number_actions).to(self.device)
    self.target_qnetwork = Network(state_size,number_actions).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(),lr=learning_rate)
    self.memory = ReplayBuffer(replay_buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

In [54]:
#initializing the Deep Q-Network
agent = Agent(state_size,number_actions)

the device available in the local system is:  cpu


In [55]:

#training the Deep Q-network agent
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break


AttributeError: 'Agent' object has no attribute 'action_size'

In [ ]:
#visualizing the lunar lander by agent
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('race_car.mp4', frames,fps=30)

show_video_of_model(agent, 'CarRacing-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()